## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-07.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.004116,-0.017328,-0.003911,-0.003210,0.009825,0.024238,-0.013265,-0.004311,0.034958,-0.013568,...,-0.002317,0.043351,-0.026372,-0.013417,-0.007547,0.004776,-0.006674,0.001050,0.020061,0.000246
2,-0.004670,0.010079,0.022950,0.003701,-0.019795,0.009395,-0.015999,-0.003027,0.053465,-0.020043,...,-0.020953,0.004588,-0.001619,-0.006128,-0.006777,0.057034,-0.002407,-0.019236,-0.009251,0.000861


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,0.00000,-0.017328,-0.003911,-0.00321,0.000000,0.0,-0.013265,-0.004311,0.0,-0.013568,...,-0.002317,0.0,-0.026372,-0.013417,-0.007547,0.0,-0.006674,0.000000,0.000000,0.0
2,-0.00467,0.000000,0.000000,0.00000,-0.019795,0.0,-0.015999,-0.003027,0.0,-0.020043,...,-0.020953,0.0,-0.001619,-0.006128,-0.006777,0.0,-0.002407,-0.019236,-0.009251,0.0


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.009649,-0.030099,-0.029500,-0.007590,0.032529,0.016301,0.003003,-0.001410,-0.020325,0.007111,...,0.020467,0.042570,-0.027184,-0.008005,-0.000845,-0.057392,-0.004687,0.022279,0.032191,-0.000675
BAJFINANCE,-0.030099,0.093891,0.092023,0.023677,-0.101473,-0.050851,-0.009367,0.004397,0.063401,-0.022181,...,-0.063844,-0.132794,0.084798,0.024972,0.002635,0.179030,0.014619,-0.069496,-0.100417,0.002107
BAJAJFINSV,-0.029500,0.092023,0.090193,0.023206,-0.099454,-0.049840,-0.009181,0.004310,0.062140,-0.021740,...,-0.062574,-0.130153,0.083112,0.024475,0.002582,0.175469,0.014328,-0.068114,-0.098419,0.002065
BRITANNIA,-0.007590,0.023677,0.023206,0.005971,-0.025589,-0.012823,-0.002362,0.001109,0.015988,-0.005593,...,-0.016100,-0.033487,0.021384,0.006297,0.000664,0.045146,0.003687,-0.017525,-0.025322,0.000531
DIVISLAB,0.032529,-0.101473,-0.099454,-0.025589,0.109667,0.054957,0.010123,-0.004752,-0.068521,0.023972,...,0.068999,0.143518,-0.091646,-0.026989,-0.002847,-0.193487,-0.015800,0.075108,0.108526,-0.002277
GRASIM,0.016301,-0.050851,-0.049840,-0.012823,0.054957,0.027541,0.005073,-0.002382,-0.034338,0.012013,...,0.034578,0.071921,-0.045927,-0.013525,-0.001427,-0.096962,-0.007918,0.037639,0.054385,-0.001141
HCLTECH,0.003003,-0.009367,-0.009181,-0.002362,0.010123,0.005073,0.000934,-0.000439,-0.006325,0.002213,...,0.006369,0.013248,-0.008460,-0.002491,-0.000263,-0.017861,-0.001458,0.006933,0.010018,-0.000210
HDFCBANK,-0.001410,0.004397,0.004310,0.001109,-0.004752,-0.002382,-0.000439,0.000206,0.002969,-0.001039,...,-0.002990,-0.006219,0.003971,0.001170,0.000123,0.008385,0.000685,-0.003255,-0.004703,0.000099
HINDALCO,-0.020325,0.063401,0.062140,0.015988,-0.068521,-0.034338,-0.006325,0.002969,0.042812,-0.014978,...,-0.043111,-0.089671,0.057261,0.016863,0.001779,0.120892,0.009872,-0.046928,-0.067808,0.001423
HINDUNILVR,0.007111,-0.022181,-0.021740,-0.005593,0.023972,0.012013,0.002213,-0.001039,-0.014978,0.005240,...,0.015083,0.031372,-0.020033,-0.005899,-0.000622,-0.042294,-0.003454,0.016418,0.023723,-0.000498


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-0.669486,0.005143,-137.139627,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-66.948551,1.227966,-54.549032,4.576064,9.676986,2.881647,3.762807,6.100055,1.147082,1.639964,...,9.762288,2.014192,7.839004,7.728703,2.277867,3.332264,1.031637,7.217097,0.606272,1.736266


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')